# Newfeatures_v4_chess
## Reference:
- https://www.kaggle.com/yezuocheng/0-780-lgbm-0-780/edit?rvi=1
- https://www.icode9.com/content-1-376184.html
- https://zhuanlan.zhihu.com/p/41422048
- http://www.huaxiaozhuan.com/%E5%B7%A5%E5%85%B7/lightgbm/chapters/lightgbm_usage.html
- https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [10]:
import pickle
import trueskill
import math
import time
import numpy as np
import random
import pandas as pd
import joblib
from collections import defaultdict
import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
# import riiideducation
from sklearn.metrics import roc_auc_score
import gc

_ = np.seterr(divide='ignore', invalid='ignore')

# Preprocess

In [11]:
data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    'task_container_id': 'int16',
    #'user_answer': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'

In [12]:
print('start read train data...')
train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()

start read train data...


In [13]:
print('start handle lecture data...')

start handle lecture data...


In [14]:
#reading in lecture df
lectures_df = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')

In [15]:
lectures_df['type_of'] = lectures_df['type_of'].replace('solving question', 'solving_question')

lectures_df = pd.get_dummies(lectures_df, columns=['part', 'type_of'])

part_lectures_columns = [column for column in lectures_df.columns if column.startswith('part')]

types_of_lectures_columns = [column for column in lectures_df.columns if column.startswith('type_of_')]

In [16]:
train_lectures = train_df[train_df.content_type_id == True].merge(lectures_df, left_on='content_id', right_on='lecture_id', how='left')

In [17]:
user_lecture_stats_part = train_lectures.groupby('user_id',as_index = False)[part_lectures_columns + types_of_lectures_columns].sum()

In [18]:
lecturedata_types_dict = {   
    'user_id': 'int32', 
    'part_1': 'int8',
    'part_2': 'int8',
    'part_3': 'int8',
    'part_4': 'int8',
    'part_5': 'int8',
    'part_6': 'int8',
    'part_7': 'int8',
    'type_of_concept': 'int8',
    'type_of_intention': 'int8',
    'type_of_solving_question': 'int8',
    'type_of_starter': 'int8'
}
user_lecture_stats_part = user_lecture_stats_part.astype(lecturedata_types_dict)

In [19]:
for column in user_lecture_stats_part.columns:
    #bool_column = column + '_boolean'
    if(column !='user_id'):
        user_lecture_stats_part[column] = (user_lecture_stats_part[column] > 0).astype('int8')

In [20]:
user_lecture_stats_part.dtypes

user_id                     int32
part_1                       int8
part_2                       int8
part_3                       int8
part_4                       int8
part_5                       int8
part_6                       int8
part_7                       int8
type_of_concept              int8
type_of_intention            int8
type_of_solving_question     int8
type_of_starter              int8
dtype: object

In [21]:
#clearing memory
del(train_lectures)
gc.collect()

136

In [22]:
user_lecture_agg = train_df.groupby('user_id')['content_type_id'].agg(['sum', 'count'])
user_lecture_agg=user_lecture_agg.astype('int16')

In [23]:

#1= if the event was the user watching a lecture.
cum = train_df.groupby('user_id')['content_type_id'].agg(['cumsum', 'cumcount'])
cum['cumcount']=cum['cumcount']+1
train_df['user_interaction_count'] = cum['cumcount'] 
train_df['user_interaction_timestamp_mean'] = train_df['timestamp']/cum['cumcount'] 
train_df['user_lecture_sum'] = cum['cumsum'] 
train_df['user_lecture_lv'] = cum['cumsum'] / cum['cumcount']


train_df.user_lecture_lv=train_df.user_lecture_lv.astype('float16')
train_df.user_lecture_sum=train_df.user_lecture_sum.astype('int16')
train_df.user_interaction_count=train_df.user_interaction_count.astype('int16')
train_df['user_interaction_timestamp_mean']=train_df['user_interaction_timestamp_mean']/(1000*3600)
train_df.user_interaction_timestamp_mean=train_df.user_interaction_timestamp_mean.astype('float32')


In [24]:
#pd.options.display.max_rows = 200

In [25]:
del cum
gc.collect()

82

In [26]:
print('start handle train_df...')

start handle train_df...


In [27]:
train_df['prior_question_had_explanation'].fillna(False, inplace=True)
train_df = train_df.astype(data_types_dict)
train_df = train_df[train_df[target] != -1].reset_index(drop=True)

In [28]:
content_explation_agg=train_df[["content_id","prior_question_had_explanation",target]].groupby(["content_id","prior_question_had_explanation"])[target].agg(['mean'])

In [29]:
content_explation_agg.dtypes

mean    float64
dtype: object

In [30]:
content_explation_agg=content_explation_agg.unstack()

content_explation_agg=content_explation_agg.reset_index()
content_explation_agg.columns = ['content_id', 'content_explation_false_mean','content_explation_true_mean']

In [31]:
content_explation_agg.content_id=content_explation_agg.content_id.astype('int16')
content_explation_agg.content_explation_false_mean=content_explation_agg.content_explation_false_mean.astype('float16')
content_explation_agg.content_explation_true_mean=content_explation_agg.content_explation_true_mean.astype('float16')

In [32]:
print('start handle attempt_no...')

start handle attempt_no...


In [33]:
train_df["attempt_no"] = 1
train_df.attempt_no=train_df.attempt_no.astype('int8')
#
attempt_no_agg=train_df.groupby(["user_id","content_id"])["attempt_no"].agg(['sum']).astype('int8')
#attempt_no_agg=attempt_no_agg.astype('int8')
train_df["attempt_no"] = train_df[["user_id","content_id",'attempt_no']].groupby(["user_id","content_id"])["attempt_no"].cumsum()

In [34]:
attempt_no_agg=attempt_no_agg[attempt_no_agg['sum'] >1]

In [35]:
print('start handle timestamp...')
prior_question_elapsed_time_mean=train_df['prior_question_elapsed_time'].mean()
train_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace=True)

start handle timestamp...


In [36]:
max_timestamp_u = train_df[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u.columns = ['user_id', 'max_time_stamp']
max_timestamp_u.user_id=max_timestamp_u.user_id.astype('int32')

In [37]:
train_df['lagtime'] = train_df.groupby('user_id')['timestamp'].shift()

max_timestamp_u2 = train_df[['user_id','lagtime']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u2.columns = ['user_id', 'max_time_stamp2']
max_timestamp_u2.user_id=max_timestamp_u2.user_id.astype('int32')

In [38]:
train_df['lagtime']=train_df['timestamp']-train_df['lagtime']
lagtime_mean=train_df['lagtime'].mean()
train_df['lagtime'].fillna(lagtime_mean, inplace=True)

In [39]:
train_df['lagtime']=train_df['lagtime']/(1000*3600)
train_df.lagtime=train_df.lagtime.astype('float32')

In [40]:
train_df['lagtime2'] = train_df.groupby('user_id')['timestamp'].shift(2)

max_timestamp_u3 = train_df[['user_id','lagtime2']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u3.columns = ['user_id', 'max_time_stamp3']
max_timestamp_u3.user_id=max_timestamp_u3.user_id.astype('int32')

train_df['lagtime2']=train_df['timestamp']-train_df['lagtime2']
lagtime_mean2=train_df['lagtime2'].mean()
train_df['lagtime2'].fillna(lagtime_mean2, inplace=True)


In [41]:
train_df['lagtime2']=train_df['lagtime2']/(1000*3600)
train_df.lagtime2=train_df.lagtime2.astype('float32')

In [42]:

train_df['lagtime3'] = train_df.groupby('user_id')['timestamp'].shift(3)

train_df['lagtime3']=train_df['timestamp']-train_df['lagtime3']
lagtime_mean3=train_df['lagtime3'].mean()
train_df['lagtime3'].fillna(lagtime_mean3, inplace=True)
train_df['lagtime3']=train_df['lagtime3']/(1000*3600)
train_df.lagtime3=train_df.lagtime3.astype('float32')

In [43]:
train_df['timestamp']=train_df['timestamp']/(1000*3600)
#
train_df.timestamp=train_df.timestamp.astype('float16')

In [44]:
user_prior_question_elapsed_time = train_df[['user_id','prior_question_elapsed_time']].groupby(['user_id']).tail(1)
user_prior_question_elapsed_time.columns = ['user_id', 'prior_question_elapsed_time']

In [45]:
train_df['question_elapsed_time'] = train_df['prior_question_elapsed_time'].shift(-1)
train_df['question_elapsed_time'] = train_df['question_elapsed_time'].astype('float16')
train_df['delta_prior_question_elapsed_time'] = train_df.groupby('user_id')['prior_question_elapsed_time'].shift()
train_df['delta_prior_question_elapsed_time']=train_df['prior_question_elapsed_time']-train_df['delta_prior_question_elapsed_time']

In [46]:
content_elapsed_time_agg=train_df.loc[train_df[target] == 1]
content_elapsed_time_agg=content_elapsed_time_agg.groupby('content_id')['question_elapsed_time'].agg(['median','mean','max','min','skew','var'])
content_elapsed_time_agg=content_elapsed_time_agg.astype('float16')
content_had_explanation_agg=train_df.groupby('content_id')['prior_question_had_explanation'].agg(['mean'])
content_had_explanation_agg=content_had_explanation_agg.astype('float16')
del train_df['question_elapsed_time']
gc.collect()

40

In [47]:
delta_prior_question_elapsed_time_mean=train_df['delta_prior_question_elapsed_time'].mean()
train_df['delta_prior_question_elapsed_time'].fillna(delta_prior_question_elapsed_time_mean, inplace=True)
train_df.delta_prior_question_elapsed_time=train_df.delta_prior_question_elapsed_time.astype('int32')

In [48]:
train_df['lag'] = train_df.groupby('user_id')[target].shift()

cum = train_df.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
##cum['cumcount']=cum['cumcount']+1
user_agg = train_df.groupby('user_id')['lag'].agg(['sum', 'count']).astype('int16')
cum['cumsum'].fillna(0, inplace=True)

train_df['user_correctness'] = cum['cumsum'] / cum['cumcount']
train_df['user_correct_count'] = cum['cumsum']
train_df['user_uncorrect_count'] = cum['cumcount']-cum['cumsum']
#train_df['user_answer_count'] = cum['cumcount']
train_df.drop(columns=['lag'], inplace=True)
train_df['user_correctness'].fillna(0.67, inplace=True)

train_df.user_correctness=train_df.user_correctness.astype('float16')
train_df.user_correct_count=train_df.user_correct_count.astype('int16')
train_df.user_uncorrect_count=train_df.user_uncorrect_count.astype('int16')
train_df['true_user_correctness'] = (train_df['user_correct_count']+1)/(train_df['user_correct_count']/train_df['user_correctness'] + 2) 
train_df.true_user_correctness=train_df.true_user_correctness.astype('float16')
#train_df.user_answer_count=train_df.user_answer_count.astype('int16')

In [49]:
del cum
gc.collect()

40

In [50]:
train_df.prior_question_had_explanation=train_df.prior_question_had_explanation.astype('int8')
explanation_agg = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['sum', 'count'])
explanation_agg=explanation_agg.astype('int16')

In [51]:
cum = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['cumsum', 'cumcount'])
cum['cumcount']=cum['cumcount']+1
train_df['explanation_mean'] = cum['cumsum'] / cum['cumcount']
train_df['explanation_true_count'] = cum['cumsum'] 
train_df['explanation_false_count'] =  cum['cumcount']-cum['cumsum']

train_df.explanation_mean=train_df.explanation_mean.astype('float16')
train_df.explanation_true_count=train_df.explanation_true_count.astype('int16')
train_df.explanation_false_count=train_df.explanation_false_count.astype('int16')

In [52]:
del cum
gc.collect()

40

In [53]:
content_agg = train_df.groupby('content_id')[target].agg(['sum', 'count','var','skew'])
task_container_agg = train_df.groupby('task_container_id')[target].agg(['sum', 'count','var'])
content_agg=content_agg.astype('float32')
task_container_agg=task_container_agg.astype('float32')

In [54]:
train_df['task_container_uncor_count'] = train_df['task_container_id'].map(task_container_agg['count']-task_container_agg['sum']).astype('int32')
train_df['task_container_cor_count'] = train_df['task_container_id'].map(task_container_agg['sum']).astype('int32')
train_df['task_container_std'] = train_df['task_container_id'].map(task_container_agg['var']).astype('float16')
train_df['task_container_correctness'] = train_df['task_container_id'].map(task_container_agg['sum'] / task_container_agg['count'])
train_df.task_container_correctness=train_df.task_container_correctness.astype('float16')

In [55]:
print('start questions data...')

start questions data...


In [56]:
questions_df = pd.read_csv(
    '../input/riiid-test-answer-prediction/questions.csv', 
    usecols=[0, 1,3,4],
    dtype={'question_id': 'int16','bundle_id': 'int16', 'part': 'int8','tags': 'str'}
)

In [58]:
que_fea_merge = pd.read_csv(
    '../input/offline-group-by-table/que_fea_merge.csv', 
    usecols=[0, 2,3,4,5,6,7,8,9],
    dtype={'content_id': 'int16','question_elapsed_time_mean': 'float16','question_had_explanation_mean': 'float16', 'question_correctly_q_count': 'int16','question_correctly_q_mean': 'float16','tags_lsi': 'float16', 'tag_acc_count': 'float16','tag_acc_max':'float16','tag_acc_min': 'float16'}
)
part_fea_merge = pd.read_csv(
    '../input/offline-group-by-table/part_fea_merge.csv', 
    usecols=[0, 1,2,3],
    dtype={'part': 'int8','part_elapsed_time_mean': 'float16','part_had_explanation_mean': 'float16','part_correctly_q_mean':'float16'}
)

In [65]:
questions_cmnts = pd.read_csv(
     '../input/riiid-sakt-base/question_cmnts (1).csv', 
     usecols=[1,2],
     dtype={'question_id': 'int16','community': 'int8'}
 )
questions_df = pd.merge(questions_df, questions_cmnts, on='question_id', how='left',right_index=True)#

In [66]:
questions_df['community'] = questions_df['community']
questions_df = pd.get_dummies(questions_df, columns=['community'])
part_questions_columns = [column for column in questions_df.columns if column.startswith('community')]

In [67]:
bundle_agg = questions_df.groupby('bundle_id')['question_id'].agg(['count'])

In [68]:
questions_df['content_sub_bundle'] = questions_df['bundle_id'].map(bundle_agg['count']).astype('int8')

In [69]:
questions_df['tags'].fillna('188', inplace=True)

In [70]:
def gettags(tags,num):
    tags_splits=tags.split(" ")
    result='' 
    for t in tags_splits:
        x=int(t)
        if(x<32*(num+1) and x>=32*num):#num 
            result=result+' '+t
    return result

In [71]:
from sklearn.preprocessing import LabelEncoder
for num in range(0,6):
    questions_df["tags"+str(num)] = questions_df["tags"].apply(lambda row: gettags(row,num))
    le = LabelEncoder()
    le.fit(np.unique(questions_df['tags'+str(num)].values))
    #questions_df[['tags'+str(num)]=
    questions_df['tags'+str(num)]=questions_df[['tags'+str(num)]].apply(le.transform)

In [72]:
questions_df_dict = {   
    'tags0': 'int8',
    'tags1': 'int8',
    'tags2': 'int8',
    'tags3': 'int8',
    'tags4': 'int8',
    'tags5': 'int8',
    #'tags6': 'int8',
    #'tags7': 'int8'
}
questions_df = questions_df.astype(questions_df_dict)

In [73]:
questions_df.drop(columns=['tags'], inplace=True)

In [74]:
questions_df['part_bundle_id']=questions_df['part']*100000+questions_df['bundle_id']
questions_df.part_bundle_id=questions_df.part_bundle_id.astype('int32')

In [77]:
questions_df.rename(columns={'question_id':'content_id'}, inplace=True)

In [78]:
questions_df = pd.merge(questions_df, que_fea_merge, on='content_id', how='left',right_index=True)
questions_df = pd.merge(questions_df, part_fea_merge, on='part', how='left',right_index=True)

In [79]:
questions_df = pd.merge(questions_df, content_explation_agg, on='content_id', how='left',right_index=True)#
# questions_df.content_explation_false_mean=questions_df.content_explation_false_mean.astype('float16')
# questions_df.content_explation_true_mean=questions_df.content_explation_true_mean.astype('float16')

In [80]:
del content_explation_agg

In [81]:
questions_df['content_uncorrect_count'] = questions_df['content_id'].map(content_agg['count']-content_agg['sum']).astype('int32')
questions_df['content_correct_count'] = questions_df['content_id'].map(content_agg['sum']).astype('int32')
questions_df['content_count'] = questions_df['content_id'].map(content_agg['count'])
questions_df.content_count=questions_df.content_count.astype('int32')

questions_df['content_correctness'] = questions_df['content_id'].map(content_agg['sum'] / content_agg['count'])
questions_df.content_correctness=questions_df.content_correctness.astype('float16')

questions_df['true_content_correctness'] = (questions_df['content_correct_count']+1)/(questions_df['content_correct_count']/questions_df['content_correctness'] + 2) 
questions_df.true_content_correctness=questions_df.true_content_correctness.astype('float16')

questions_df['content_correctness_skew'] = questions_df['content_id'].map(content_agg['skew'])
questions_df.content_correctness_skew=questions_df.content_correctness_skew.astype('float16')

questions_df['content_correctness_std'] = questions_df['content_id'].map(content_agg['var'])
questions_df.content_correctness_std=questions_df.content_correctness_std.astype('float16')

In [82]:
questions_df['content_max_time'] = questions_df['content_id'].map(content_elapsed_time_agg['max'])
questions_df.content_max_time=questions_df.content_max_time.astype('float16')
questions_df['content_min_time'] = questions_df['content_id'].map(content_elapsed_time_agg['min'])
questions_df.content_min_time=questions_df.content_min_time.astype('float16')

questions_df['content_med_time'] = questions_df['content_id'].map(content_elapsed_time_agg['median'])
questions_df.content_med_time=questions_df.content_med_time.astype('float16')
questions_df['part_med_time'] = questions_df.groupby('part')['content_med_time'].agg(['mean'])
questions_df['part_med_time'] = questions_df['part_med_time'].astype('float16')

questions_df['content_time_std'] = questions_df['content_id'].map(content_elapsed_time_agg['var'])
questions_df.content_time_std=questions_df.content_time_std.astype('float16')
questions_df['part_time_std'] = questions_df.groupby('part')['content_time_std'].agg(['mean'])
questions_df['part_time_std'] = questions_df['part_time_std'].astype('float16')

questions_df['content_time_skew'] = questions_df['content_id'].map(content_elapsed_time_agg['skew'])
questions_df.content_time_skew=questions_df.content_time_skew.astype('float16')
questions_df['part_time_skew'] = questions_df.groupby('part')['content_time_skew'].agg(['mean'])
questions_df['part_time_skew'] = questions_df['part_time_skew'].astype('float16')

questions_df['content_mean_time'] = questions_df['content_id'].map(content_elapsed_time_agg['mean'])
questions_df['content_mean_time'] = questions_df['content_mean_time'].astype('float16')
questions_df['part_mean_time'] = questions_df.groupby('part')['content_mean_time'].agg(['mean'])
questions_df['part_mean_time'] = questions_df['part_mean_time'].astype('float16')

questions_df['mean_time_standard'] = questions_df['content_mean_time']/questions_df['part_mean_time']
questions_df['mean_time_standard'] = questions_df['mean_time_standard'].astype('float16')

questions_df['content_time_range'] = questions_df['content_max_time']-questions_df['content_min_time']
questions_df.content_time_range=questions_df.content_time_range.astype('float16')

questions_df['content_had_explanation_mean'] = questions_df['content_id'].map(content_had_explanation_agg['mean'])
questions_df.content_had_explanation_mean=questions_df.content_had_explanation_mean.astype('float16')

In [83]:
del content_elapsed_time_agg
del content_had_explanation_agg
gc.collect()

219

In [84]:
part_agg = questions_df.groupby('part')['content_correctness'].agg(['mean', 'var'])
questions_df['part_correctness_mean'] = questions_df['part'].map(part_agg['mean'])
questions_df['part_correctness_std'] = questions_df['part'].map(part_agg['var'])
questions_df.part_correctness_mean=questions_df.part_correctness_mean.astype('float16')
questions_df.part_correctness_std=questions_df.part_correctness_std.astype('float16')

In [85]:
part_agg = questions_df.groupby('part')['content_uncorrect_count'].agg(['sum'])
questions_df['part_uncor_count'] = questions_df['part'].map(part_agg['sum']).astype('int32')
#
part_agg = questions_df.groupby('part')['content_correct_count'].agg(['sum'])
questions_df['part_cor_count'] = questions_df['part'].map(part_agg['sum']).astype('int32')

In [86]:
bundle_agg = questions_df.groupby('bundle_id')['content_correctness'].agg(['mean'])
questions_df['bundle_correctness_mean'] = questions_df['bundle_id'].map(bundle_agg['mean'])
questions_df.bundle_correctness_mean=questions_df.bundle_correctness_mean.astype('float16')

In [87]:
tags1_agg = questions_df.groupby('tags0')['mean_time_standard'].agg(['skew'])
questions_df['tags0_mean_time'] = questions_df['tags0'].map(tags1_agg['skew'])
questions_df.tags0_mean_time=questions_df.tags0_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags0')['mean_time_standard'].agg(['var'])
questions_df['tags0_med_time'] = questions_df['tags0'].map(tags1_agg['var'])
questions_df.tags0_med_time=questions_df.tags0_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags0')['content_correctness'].agg(['mean'])
questions_df['tags0_correctness_mean'] = questions_df['tags0'].map(tags1_agg['mean'])
questions_df.tags0_correctness_mean=questions_df.tags0_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags0')['content_correctness'].agg(['skew'])
questions_df['tags0_correctness_skew'] = questions_df['tags0'].map(tags1_agg['skew'])
questions_df.tags0_correctness_skew=questions_df.tags0_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags0')['content_correctness'].agg(['var'])
questions_df['tags0_correctness_var'] = questions_df['tags0'].map(tags1_agg['var'])
questions_df.tags0_correctness_var=questions_df.tags0_correctness_var.astype('float16')


tags1_agg = questions_df.groupby('tags1')['mean_time_standard'].agg(['skew'])
questions_df['tags1_mean_time'] = questions_df['tags1'].map(tags1_agg['skew'])
questions_df.tags1_mean_time=questions_df.tags1_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags1')['mean_time_standard'].agg(['var'])
questions_df['tags1_med_time'] = questions_df['tags1'].map(tags1_agg['var'])
questions_df.tags1_med_time=questions_df.tags1_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags1')['content_correctness'].agg(['mean'])
questions_df['tags1_correctness_mean'] = questions_df['tags1'].map(tags1_agg['mean'])
questions_df.tags1_correctness_mean=questions_df.tags1_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags1')['content_correctness'].agg(['skew'])
questions_df['tags1_correctness_skew'] = questions_df['tags1'].map(tags1_agg['skew'])
questions_df.tags1_correctness_skew=questions_df.tags1_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags1')['content_correctness'].agg(['var'])
questions_df['tags1_correctness_var'] = questions_df['tags1'].map(tags1_agg['var'])
questions_df.tags1_correctness_var=questions_df.tags1_correctness_var.astype('float16')


tags1_agg = questions_df.groupby('tags2')['mean_time_standard'].agg(['skew'])
questions_df['tags2_mean_time'] = questions_df['tags2'].map(tags1_agg['skew'])
questions_df.tags2_mean_time=questions_df.tags2_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags2')['mean_time_standard'].agg(['var'])
questions_df['tags2_med_time'] = questions_df['tags2'].map(tags1_agg['var'])
questions_df.tags2_med_time=questions_df.tags2_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags2')['content_correctness'].agg(['mean'])
questions_df['tags2_correctness_mean'] = questions_df['tags2'].map(tags1_agg['mean'])
questions_df.tags2_correctness_mean=questions_df.tags2_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags2')['content_correctness'].agg(['skew'])
questions_df['tags2_correctness_skew'] = questions_df['tags2'].map(tags1_agg['skew'])
questions_df.tags2_correctness_skew=questions_df.tags2_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags2')['content_correctness'].agg(['var'])
questions_df['tags2_correctness_var'] = questions_df['tags2'].map(tags1_agg['var'])
questions_df.tags2_correctness_var=questions_df.tags2_correctness_var.astype('float16')


tags1_agg = questions_df.groupby('tags3')['mean_time_standard'].agg(['skew'])
questions_df['tags3_mean_time'] = questions_df['tags3'].map(tags1_agg['skew'])
questions_df.tags3_mean_time=questions_df.tags3_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags3')['mean_time_standard'].agg(['var'])
questions_df['tags3_med_time'] = questions_df['tags3'].map(tags1_agg['var'])
questions_df.tags3_med_time=questions_df.tags3_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags3')['content_correctness'].agg(['mean'])
questions_df['tags3_correctness_mean'] = questions_df['tags3'].map(tags1_agg['mean'])
questions_df.tags3_correctness_mean=questions_df.tags3_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags3')['content_correctness'].agg(['skew'])
questions_df['tags3_correctness_skew'] = questions_df['tags3'].map(tags1_agg['skew'])
questions_df.tags3_correctness_skew=questions_df.tags3_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags3')['content_correctness'].agg(['var'])
questions_df['tags3_correctness_var'] = questions_df['tags3'].map(tags1_agg['var'])
questions_df.tags3_correctness_var=questions_df.tags3_correctness_var.astype('float16')


tags1_agg = questions_df.groupby('tags4')['mean_time_standard'].agg(['skew'])
questions_df['tags4_mean_time'] = questions_df['tags4'].map(tags1_agg['skew'])
questions_df.tags4_mean_time=questions_df.tags4_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags4')['mean_time_standard'].agg(['var'])
questions_df['tags4_med_time'] = questions_df['tags4'].map(tags1_agg['var'])
questions_df.tags4_med_time=questions_df.tags4_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags4')['content_correctness'].agg(['mean'])
questions_df['tags4_correctness_mean'] = questions_df['tags4'].map(tags1_agg['mean'])
questions_df.tags4_correctness_mean=questions_df.tags4_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags4')['content_correctness'].agg(['skew'])
questions_df['tags4_correctness_skew'] = questions_df['tags4'].map(tags1_agg['skew'])
questions_df.tags4_correctness_skew=questions_df.tags4_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags4')['content_correctness'].agg(['var'])
questions_df['tags4_correctness_var'] = questions_df['tags4'].map(tags1_agg['var'])
questions_df.tags4_correctness_var=questions_df.tags4_correctness_var.astype('float16')


tags1_agg = questions_df.groupby('tags5')['mean_time_standard'].agg(['skew'])
questions_df['tags5_mean_time'] = questions_df['tags5'].map(tags1_agg['skew'])
questions_df.tags5_mean_time=questions_df.tags5_mean_time.astype('float16')
tags1_agg = questions_df.groupby('tags5')['mean_time_standard'].agg(['var'])
questions_df['tags5_med_time'] = questions_df['tags5'].map(tags1_agg['var'])
questions_df.tags5_med_time=questions_df.tags5_med_time.astype('float16')

tags1_agg = questions_df.groupby('tags5')['content_correctness'].agg(['mean'])
questions_df['tags5_correctness_mean'] = questions_df['tags5'].map(tags1_agg['mean'])
questions_df.tags5_correctness_mean=questions_df.tags5_correctness_mean.astype('float16')
tags1_agg = questions_df.groupby('tags5')['content_correctness'].agg(['skew'])
questions_df['tags5_correctness_skew'] = questions_df['tags5'].map(tags1_agg['skew'])
questions_df.tags5_correctness_skew=questions_df.tags5_correctness_skew.astype('float16')
tags1_agg = questions_df.groupby('tags5')['content_correctness'].agg(['var'])
questions_df['tags5_correctness_var'] = questions_df['tags5'].map(tags1_agg['var'])
questions_df.tags5_correctness_var=questions_df.tags5_correctness_var.astype('float16')

In [89]:
del content_agg
del bundle_agg
del part_agg
del tags1_agg
del train_df
gc.collect()

80

# Train

In [90]:
features_dict = {
    #'user_id',
    #'timestamp':'float16',#
    'user_interaction_count':'int16',
    'user_interaction_timestamp_mean':'float32',
    'lagtime':'float32',#
    'lagtime2':'float32',
    'lagtime3':'float32',
    #'lagtime_mean':'int32',
    #'content_id':'int16',
    #'task_container_id':'int16',
    'user_lecture_sum':'int16',#
    'user_lecture_lv':'float16',##
    'prior_question_elapsed_time':'float32',#
    #'delta_prior_question_elapsed_time':'int32',#
    'user_correctness':'float16',#
    'true_user_correctness':'float16',
    'user_uncorrect_count':'int16',#
    'user_correct_count':'int16',#
    'content_correctness':'float16',
    'true_content_correctness':'float16',
    'content_correctness_std':'float16',
    'content_count':'int32',
    'content_correct_count':'int32',
    'content_uncorrect_count':'int32',#
    'content_correctness_skew':'float16',
    #'content_time_std':'float16',
    #'part_time_std':'float16',
    #'content_time_skew':'float16',
    #'part_time_skew':'float16',
    'content_mean_time':'float16',
    'part_mean_time':'float16',
    'mean_time_standard':'float16',
    'content_med_time':'float16',
    'part_med_time':'float16',
    'content_time_range':'float16',
    #'content_elapsed_time_mean':'float16',
    'content_had_explanation_mean':'float16',
    'content_explation_false_mean':'float16',
    'content_explation_true_mean':'float16',
    'task_container_correctness':'float16',
    #'task_container_std':'float16',
    'task_container_cor_count':'int32',#
    'task_container_uncor_count':'int32',#
    'attempt_no':'int8',#
    #'part':'int8',
    'part_correctness_mean':'float16',
    'part_correctness_std':'float16',
    'part_uncor_count':'int32',
    'part_cor_count':'int32',
    #'tags0': 'int8',
    #'tags1': 'int8',
    #'tags2': 'int8',
    #'tags3': 'int8',
    #'tags4': 'int8',
    #'tags5': 'int8',
    #'tags6': 'int8',
    #'tags7': 'int8',
     #'tags0_correctness_mean':'float16',
     #'tags1_correctness_mean':'float16',
     #'tags2_correctness_mean':'float16',
     #'tags3_correctness_mean':'float16',
     #'tags4_correctness_mean':'float16',
     #'tags5_correctness_mean':'float16',
     'tags0_correctness_skew':'float16',
     'tags1_correctness_skew':'float16',
     'tags2_correctness_skew':'float16',
     'tags3_correctness_skew':'float16',
     'tags4_correctness_skew':'float16',
     'tags5_correctness_skew':'float16',
     #'tags0_correctness_var':'float16',
     #'tags1_correctness_var':'float16',
     #'tags2_correctness_var':'float16',
     #'tags3_correctness_var':'float16',
     #'tags4_correctness_var':'float16',
     #'tags5_correctness_var':'float16',
    'question_elapsed_time_mean': 'float16',
    'question_had_explanation_mean': 'float16',
    'question_correctly_q_count': 'int16',
    'question_correctly_q_mean': 'float16',
    'tags_lsi': 'float16', 
    'tag_acc_count': 'float16',	
    'tag_acc_max':'float16',
    'tag_acc_min': 'float16',
    'part_elapsed_time_mean': 'float16',
    'part_had_explanation_mean': 'float16',
    'part_correctly_q_mean':'float16',
#     'bundle_id':'int16',
#     'bundle_correctness_mean':'float16',
#     'bundle_uncor_count':'int32',
#     'bundle_cor_count':'int32',
    #'part_bundle_id':'int32',
    'content_sub_bundle':'int8',
    'prior_question_had_explanation':'int8',
    'explanation_mean':'float16', #
    #'explanation_var',#
    'explanation_false_count':'int16',#
    'explanation_true_count':'int16',#
    'community_0':'int8',
    'community_1':'int8',
    'community_2':'int8',
    'community_3':'int8',
    'community_4':'int8',
   # 'community':'int8',
     #'parts_1':'int8',
     #'parts_2':'int8',
     #'parts_3':'int8',
     #'parts_4':'int8',
     #'parts_5':'int8',
     #'parts_6':'int8',
     #'parts_7':'int8',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
}
categorical_columns= [
    #'user_id',
    #'content_id',
    #'task_container_id',
    #'part',
   # 'community',
   #'tags0',
    #'tags1',
    #'tags2',
    #'tags3',
    #'tags4',
    #'tags5',
    #'tags6',
    #'tags7',
    #'bundle_id',
    #'part_bundle_id',
    'content_sub_bundle',
    'prior_question_had_explanation',
    'community_0',
    'community_1',
    'community_2',
    'community_3',
    'community_4', 
     #'parts_1',
     #'parts_2',
     #'parts_3',
     #'parts_4',
     #'parts_5',
     #'parts_6',
     #'parts_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
]

features=list(features_dict.keys())

In [92]:
model = joblib.load('../data/bagging4.3.pkl')

In [94]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=MAX_SEQ): 
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills)
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_          
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        return x, questions

In [95]:
user_sum_dict = user_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_count_dict = user_agg['count'].astype('int16').to_dict(defaultdict(int))
# user_sum_dict2 = user_agg2['sum'].astype('int16').to_dict(defaultdict(int))
# user_count_dict2 = user_agg2['count'].astype('int16').to_dict(defaultdict(int))

In [96]:
#user_var_dict = user_agg['var'].astype('float16').to_dict(defaultdict(int))

# content_sum_dict = content_agg['sum'].astype('int32').to_dict(defaultdict(int))
# content_count_dict = content_agg['count'].astype('int32').to_dict(defaultdict(int))

del user_agg
#del user_agg2
#del content_agg
gc.collect()

task_container_sum_dict = task_container_agg['sum'].astype('int32').to_dict(defaultdict(int))
task_container_count_dict = task_container_agg['count'].astype('int32').to_dict(defaultdict(int))
task_container_std_dict = task_container_agg['var'].astype('float16').to_dict(defaultdict(int))

explanation_sum_dict = explanation_agg['sum'].astype('int16').to_dict(defaultdict(int))
explanation_count_dict = explanation_agg['count'].astype('int16').to_dict(defaultdict(int))
#explanation_var_dict = explanation_agg['var'].astype('float16').to_dict(defaultdict(int))
del task_container_agg
del explanation_agg
gc.collect()

0

In [97]:
user_lecture_sum_dict = user_lecture_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_lecture_count_dict = user_lecture_agg['count'].astype('int16').to_dict(defaultdict(int))

#lagtime_mean_dict = lagtime_agg['mean'].astype('int32').to_dict(defaultdict(int))
#del prior_question_elapsed_time_agg
del user_lecture_agg
#del lagtime_agg
gc.collect()

95

In [98]:
max_timestamp_u_dict=max_timestamp_u.set_index('user_id').to_dict()
max_timestamp_u_dict2=max_timestamp_u2.set_index('user_id').to_dict()
max_timestamp_u_dict3=max_timestamp_u3.set_index('user_id').to_dict()
user_prior_question_elapsed_time_dict=user_prior_question_elapsed_time.set_index('user_id').to_dict()
#del question_elapsed_time_agg
del max_timestamp_u
del max_timestamp_u2
del max_timestamp_u3
del user_prior_question_elapsed_time
gc.collect()

95

In [99]:
attempt_no_sum_dict = attempt_no_agg['sum'].to_dict(defaultdict(int))

del attempt_no_agg
gc.collect()

0

In [100]:
def get_max_attempt(user_id,content_id):
    k = (user_id,content_id)

    if k in attempt_no_sum_dict.keys():
        attempt_no_sum_dict[k]+=1
        return attempt_no_sum_dict[k]

    attempt_no_sum_dict[k] = 1
    return attempt_no_sum_dict[k]

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()
prior_test_df = None
prev_test_df1 = None

In [ ]:
N=[0.4,0.6]

In [ ]:
# 路径
question_metadata_dir = r'../input/riiid-dataset/question_metadata.csv'
lesson_metadata_dir = r'../input/riiid-dataset/lesson_metadata.csv'
pickle_dir = r'..input/riiid-dataset/stage.pickle'
model_dir = r'../input/riiid-dataset/classifier.model'

env_1 = trueskill.TrueSkill(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164, draw_probability=0)
env_1.make_as_global()


def win_probability(team1, team2):
    '''
    根据两个TrueSkill对象，计算获胜概率
    :param team1:用户TrueSkill对象
    :param team2:问题Trueskill对象
    :return: 获胜概率
    '''
    delta_mu = team1.mu - team2.mu
    sum_sigma = sum([team1.sigma ** 2, team2.sigma ** 2])
    size = 2
    denom = math.sqrt(size * (0.05 * 0.05) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)


class user:
    '''
    用户 类
    '''

    def __init__(self):
        '''
        初始化user类
        :param None
        :return: None
        '''
        # 直接可输出的特征

        # 数量类
        self.question_answered_num = 0  # 用户回答问题的总数量
        self.question_answered_num_agg_field = [0] * 7  # 用户总回答的问题(按TOEIC学科领域统计)

        # 正确率类
        self.question_answered_mean_accuracy = 0  # 用户回答的问题的平均正确率
        self.question_answered_mean_accuracy_agg_field = [0] * 7  # 用户总回答的问题的平均正确率
        self.question_answered_mean_difficulty_weighted_accuracy = 0  # 用户总回答的问题的平均难度加权正确率
        self.question_answered_mean_difficulty_weighted_accuracy_agg_field = [0] * 7  # 用户总回答的问题的平均难度加权正确率(按TOEIC学科领域统计)

        # 极值类
        self.max_solved_difficulty = 1  # 用户解答的最难问题
        self.max_solved_difficulty_agg_field = [1] * 7  # 用户解答的最难问题(按TOEIC学科领域统计)
        self.min_wrong_difficulty = 0  # 用户做错的最简单问题
        self.min_wrong_difficulty_agg_field = [0] * 7  # 用户做错的最简单问题

        # 课程学习类
        self.lessons_overall = 0  # 用户总共学了多少课
        self.lessons_overall_agg_field = [0] * 7  # 用户总共学了多少课（按TOEIC学科领域统计）

        # 交互时间信息类
        self.session_time = 0  # 用户本Session的分钟数
        self.since_last_session_time = 0  # 距离上次Session的小时数

        # 需要进一步处理的特征
        self._mmr_object = trueskill.setup(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164,
                                           draw_probability=0).Rating()  # MMR模块
        self._mmr_object_agg_field = [trueskill.setup(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164,
                                                      draw_probability=0).Rating()] * 7  # MMR模块（按TOEIC学科领域统计）
        self._most_liked_guess = [0] * 4  # 用户做错时最喜欢的选项
        self._last_session_start_time = 0  # 本Session开始的时间
        self._first_action_time = 0  # 首次交互的时间
        self._question_num_dict = {}  # 用户回答问题的记录
        self._first_processed_flag = False  # 是否处理的表示

    def update_user(self, data: pd.DataFrame):
        '''
        处理一帧测试集
        :param data: pandas DataFrame
        :return: None
        '''
        _temp = None

        # 判断用户是否正在观看课程
        if data['content_type_id'] == 0:
            # Content Type 为 0，即用户正在回答问题

            # 处理回答计数部分
            self.question_answered_num = self.question_answered_num + 1
            question_field = int(data['content_field'])
            self.question_answered_num_agg_field[question_field - 1] = int(self.question_answered_num_agg_field[
                                                                               question_field - 1]) + 1

            # 处理正确率部分
            if data['answered_correctly'] == 1:
                self.question_answered_mean_accuracy = \
                    (self.question_answered_mean_accuracy * (
                            self.question_answered_num - 1) + 1) / self.question_answered_num

                self.question_answered_mean_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1) + 1) \
                    / self.question_answered_num_agg_field[question_field - 1]

                self.question_answered_mean_difficulty_weighted_accuracy = \
                    (self.question_answered_mean_difficulty_weighted_accuracy * (self.question_answered_num - 1) + (
                            1 - data['mean_question_accuracy']) * 3) \
                    / self.question_answered_num

                self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1) + (
                             1 - data['mean_question_accuracy']) * 3) \
                    / self.question_answered_num_agg_field[question_field - 1]


            else:
                self.question_answered_mean_accuracy = \
                    (self.question_answered_mean_accuracy * (
                            self.question_answered_num - 1)) / self.question_answered_num

                self.question_answered_mean_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1)) / \
                    self.question_answered_num_agg_field[question_field - 1]

                self.question_answered_mean_difficulty_weighted_accuracy = \
                    (self.question_answered_mean_difficulty_weighted_accuracy * (self.question_answered_num - 1)) \
                    / self.question_answered_num

                self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1)) \
                    / self.question_answered_num_agg_field[question_field - 1]

            # 处理最大/最小正确率部分

            if data['answered_correctly'] == 1:
                if data['mean_question_accuracy'] < self.max_solved_difficulty:
                    self.max_solved_difficulty = data['mean_question_accuracy']
                if data['mean_question_accuracy'] < self.max_solved_difficulty_agg_field[question_field - 1]:
                    self.max_solved_difficulty_agg_field[question_field - 1] = data['mean_question_accuracy']
            else:
                if data['mean_question_accuracy'] > self.min_wrong_difficulty:
                    self.min_wrong_difficulty = data['mean_question_accuracy']
                if data['mean_question_accuracy'] > self.min_wrong_difficulty_agg_field[question_field - 1]:
                    self.min_wrong_difficulty_agg_field[question_field - 1] = data['mean_question_accuracy']

            # 处理猜测部分
            if data['answered_correctly'] == 0:
                self._most_liked_guess[int(data['user_answer'])] = self._most_liked_guess[
                                                                       int(data['user_answer'])] + 1

            # 处理时间部分
            if self._first_action_time == 0:
                self._first_action_time = data['timestamp']
                self._last_session_start_time = data['timestamp']
            else:
                if data['timestamp'] - self._last_session_start_time >= 7200 * 1000:
                    self.since_last_session_time = (data[
                                                        'timestamp'] - self._last_session_start_time) / 1000 / 3600
                    self._last_session_start_time = data['timestamp']
                    self.session_time = 0
                else:
                    self.session_time = (data['timestamp'] - self._last_session_start_time) / 1000 / 60

            # 处理问题记录部分
            if str(data['content_id']) in self._question_num_dict:
                self._question_num_dict[str(data['content_id'])] = self._question_num_dict[str(data['content_id'])] + 1
            else:
                self._question_num_dict[str(data['content_id'])] = 1

            # 处理TrueSkill部分
            if data['answered_correctly'] == 1:
                self._mmr_object, _temp = \
                    trueskill.rate_1vs1(self._mmr_object,
                                        trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05, tau=0.00164, draw_probability=0).Rating())
                self._mmr_object_agg_field[question_field - 1], _temp = \
                    trueskill.rate_1vs1(self._mmr_object_agg_field[question_field - 1],
                                        trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05,
                                                        tau=0.00164, draw_probability=0).Rating())
            else:
                _temp, self._mmr_object = \
                    trueskill.rate_1vs1(trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05, tau=0.00164, draw_probability=0).Rating(),
                                        self._mmr_object)

                _temp, self._mmr_object_agg_field[question_field - 1] = \
                    trueskill.rate_1vs1(trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05,
                                                        tau=0.00164, draw_probability=0).Rating(),
                                        self._mmr_object_agg_field[question_field - 1])



        else:
            # Content Type 不为 0 ，即用户在观看视频

            self.lessons_overall = self.lessons_overall + 1
            lesson_field = int(data['content_field'])
            self.lessons_overall_agg_field[lesson_field - 1] = self.lessons_overall_agg_field[lesson_field - 1] + 1

    def process_output(self, data):
        '''
        根据user现有属性设置输出训练数据
        :param data: 本行数据集
        :return: output_dict 训练数据
        '''
        output_dict = {}

        # 回答数量类
        output_dict['question_answered_num'] = self.question_answered_num
        output_dict['question_answered_num_agg_field'] = self.question_answered_num_agg_field[
            int(data['content_field']) - 1]

        # 回答正确率类
        output_dict['question_answered_mean_accuracy'] = self.question_answered_mean_accuracy

        output_dict['question_answered_mean_accuracy_agg_field'] = self.question_answered_mean_accuracy_agg_field[
            int(data['content_field']) - 1]
        output_dict[
            'question_answered_mean_difficulty_weighted_accuracy'] = self.question_answered_mean_difficulty_weighted_accuracy
        output_dict['question_answered_mean_difficulty_weighted_accuracy_agg_field'] = \
            self.question_answered_mean_difficulty_weighted_accuracy_agg_field[int(data['content_field']) - 1]

        # 极值类

        output_dict['max_solved_difficulty'] = self.max_solved_difficulty
        output_dict['max_solved_difficulty_agg_field'] = self.max_solved_difficulty_agg_field[
            int(data['content_field']) - 1]
        output_dict['min_wrong_difficulty'] = self.min_wrong_difficulty
        output_dict['min_wrong_difficulty_agg_field'] = self.min_wrong_difficulty_agg_field[
            int(data['content_field']) - 1]

        # 课程学习类
        output_dict['lessons_overall'] = self.lessons_overall
        output_dict['lessons_overall_agg_field'] = self.lessons_overall_agg_field[int(data['content_field']) - 1]
        if output_dict['lessons_overall_agg_field'] > 0:
            output_dict['field_learnt'] = 1
        else:
            output_dict['field_learnt'] = 0
        # 交互时间类
        output_dict['session_time'] = self.session_time
        output_dict['time_to_last_session'] = self.since_last_session_time

        output_dict['task_id'] = data['task_container_id']
        output_dict['prior_time'] = data['prior_question_elapsed_time']
        # 问题信息类
        output_dict['mean_question_accuracy'] = data['mean_question_accuracy']
        output_dict['std_question_accuracy'] = data['std_accuracy']
        output_dict['question_id'] = data['content_id']
        # TrueSkill 信息类
        output_dict['mmr_overall'] = self._mmr_object.mu
        output_dict['mmr_overall_agg_field'] = self._mmr_object_agg_field[int(data['content_field']) - 1].mu
        output_dict['mmr_confidence'] = self._mmr_object.sigma

        output_dict['mmr_overall_agg_field'] = self._mmr_object_agg_field[int(data['content_field']) - 1].sigma
        output_dict['mmr_win_prob'] = win_probability(self._mmr_object,
                                                      trueskill.setup(mu=1 - data['mean_question_accuracy'],
                                                                      sigma=0.164486,
                                                                      beta=0.05, tau=0.00164,
                                                                      draw_probability=0).Rating())
        output_dict['mmr_win_prob_agg_field'] = win_probability(
            self._mmr_object_agg_field[int(data['content_field']) - 1],
            trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486, beta=0.05,
                            tau=0.00164, draw_probability=0).Rating())
        output_dict['user_id'] = data['user_id']
        output_dict['tag_1'] = data['tag_1']
        output_dict['tag_2'] = data['tag_2']

        output_dict['tags_encoded'] = data['tags_encoded']
        # 特殊特征类

        if not pd.isna(['prior_question_had_explanation']):
            output_dict['previous_explained'] = data['prior_question_had_explanation']
        else:
            output_dict['previous_explained'] = False

        if str(data['content_id']) in self._question_num_dict:
            output_dict['question_seen'] = 1
        else:
            output_dict['question_seen'] = 0

        # 猜测类
        max_choice = 0
        max_choice_num = 0
        i = 0
        for item in self._most_liked_guess:
            if item > max_choice_num:
                max_choice_num = item
                max_choice = i
            i = i + 1

        if output_dict['mmr_win_prob'] <= 0.4:
            if max_choice == data['correct_answer']:
                output_dict['most_liked_guess_correct'] = True
            else:
                output_dict['most_liked_guess_correct'] = False
        else:
            output_dict['most_liked_guess_correct'] = True

        # 训练目标
        # output_dict['answered_correctly'] = data['answered_correctly']

        return output_dict


# 导入Metadata
question_metadata = pd.read_csv(question_metadata_dir)
lesson_metadata = pd.read_csv(lesson_metadata_dir)


# 设置Metadata索引
question_metadata = question_metadata.set_index(keys=['content_id'])
lesson_metadata = lesson_metadata.set_index(keys=['content_id'])


# 导入Pickle状态
with open(pickle_dir, 'rb') as fo:
    user_pickle = pickle.load(fo)



# 重置Trueskill状态
for user_id, user_info in user_pickle.items():
    user_pickle[user_id]._mmr_object = trueskill.setup(mu=user_pickle[user_id]._mmr_object[0],
                                                       sigma=user_pickle[user_id]._mmr_object[1],
                                                       beta=0.05, tau=0.00164,
                                                       draw_probability=0).Rating()
    for i in range(0, 7):
        # 1+1
        user_pickle[user_id]._mmr_object_agg_field[i] = trueskill.setup(
            mu=user_pickle[user_id]._mmr_object_agg_field[i][0],
            sigma=user_pickle[user_id]._mmr_object_agg_field[i][1],
            beta=0.05, tau=0.00164,
            draw_probability=0).Rating()



# 导入模型
model_chess = lgb.Booster(model_file=model_dir)


# 初始换变量
rows_accum = 0  # 行计数器
first_submission = True  # 是否第一组标记
model_prd = [0]
true_value = []
last_df = pd.DataFrame()

In [ ]:
%%time

for (test_df, sample_prediction_df) in iter_test:
    test_df1=test_df.copy()
    if (prev_test_df1 is not None):
        prev_test_df1['answered_correctly'] = eval(test_df1['prior_group_answers_correct'].iloc[0])
        prev_test_df1 = prev_test_df1[prev_test_df1.content_type_id == False]
        
        prev_group = prev_test_df1[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))
        for prev_user_id in prev_group.index:
            if prev_user_id in group.index:
                group[prev_user_id] = (
                    np.append(group[prev_user_id][0], prev_group[prev_user_id][0])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][1], prev_group[prev_user_id][1])[-MAX_SEQ:]
                )
 
            else:
                group[prev_user_id] = (
                    prev_group[prev_user_id][0], 
                    prev_group[prev_user_id][1]
                )

    prev_test_df1 = test_df1.copy()
    
    test_df1 = test_df1[test_df1.content_type_id == False]
    test_dataset = TestDataset(group, test_df1, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in test_dataloader:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        with torch.no_grad():
            output, att_weight = nn_model(x, target_id)
        outs.extend(torch.sigmoid(output)[:, -1].view(-1).data.cpu().numpy())
        
    #test_df1['answered_correctly'] = outs
    
    
    
    
    ###for lgb
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[target] != -1].reset_index(drop=True)
        #prior_test_df = prior_test_df[prior_test_df[target] != -1]
        prior_test_df['prior_question_had_explanation'].fillna(False, inplace=True)       
        prior_test_df.prior_question_had_explanation=prior_test_df.prior_question_had_explanation.astype('int8')
    
        user_ids = prior_test_df['user_id'].values
        #content_ids = prior_test_df['content_id'].values
        #task_container_ids = prior_test_df['task_container_id'].values
        #prior_question_had_explanations = prior_test_df['prior_question_had_explanation'].values
        targets = prior_test_df[target].values        
        
        for user_id, answered_correctly in zip(user_ids,targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
#             user_sum_dict2[user_id] += answered_correctly
#             user_count_dict2[user_id] += 1   
            

    prior_test_df = test_df.copy() 
        
    
    question_len=len( test_df[test_df['content_type_id'] == 0])
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df.prior_question_had_explanation=test_df.prior_question_had_explanation.astype('int8')
    test_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace=True)
    

    user_lecture_sum = np.zeros(question_len, dtype=np.int16)
    user_lecture_count = np.zeros(question_len, dtype=np.int16) 
    
    user_sum = np.zeros(question_len, dtype=np.int16)
    user_count = np.zeros(question_len, dtype=np.int16)
#     user_sum2 = np.zeros(question_len, dtype=np.int16)
#     user_count2 = np.zeros(question_len, dtype=np.int16)

#     user_sum_dict_test=defaultdict(int)
#     user_count_dict_test=defaultdict(int)

    task_container_sum = np.zeros(question_len, dtype=np.int32)
    task_container_count = np.zeros(question_len, dtype=np.int32)
    task_container_std = np.zeros(question_len, dtype=np.float16)

    explanation_sum = np.zeros(question_len, dtype=np.int32)
    explanation_count = np.zeros(question_len, dtype=np.int32)
    delta_prior_question_elapsed_time = np.zeros(question_len, dtype=np.int32)

    attempt_no_count = np.zeros(question_len, dtype=np.int16)
    lagtime = np.zeros(question_len, dtype=np.float32)
    lagtime2 = np.zeros(question_len, dtype=np.float32)
    lagtime3 = np.zeros(question_len, dtype=np.float32)
    #lagtime_means = np.zeros(question_len, dtype=np.int32)
    #
   
    i=0
    for j, (user_id,prior_question_had_explanation,content_type_id,prior_question_elapsed_time,timestamp, content_id,task_container_id) in enumerate(zip(test_df['user_id'].values,test_df['prior_question_had_explanation'].values,test_df['content_type_id'].values,test_df['prior_question_elapsed_time'].values,test_df['timestamp'].values, test_df['content_id'].values, test_df['task_container_id'].values)):
        
         #
        user_lecture_sum_dict[user_id] += content_type_id
        user_lecture_count_dict[user_id] += 1
        if(content_type_id==1):#
            x=1
#             if(len(user_lecture_stats_part[user_lecture_stats_part.user_id==user_id])==0):
#                 user_lecture_stats_part = user_lecture_stats_part.append([{'user_id':user_id}], ignore_index=True)
#                 user_lecture_stats_part.fillna(0, inplace=True)
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
#             else:
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
        if(content_type_id==0):#   
            user_lecture_sum[i] = user_lecture_sum_dict[user_id]
            user_lecture_count[i] = user_lecture_count_dict[user_id]
                
            user_sum[i] = user_sum_dict[user_id]
            user_count[i] = user_count_dict[user_id]
#             user_sum2[i] = user_sum_dict2[user_id]
#             user_count2[i] = user_count_dict2[user_id]
    #         content_sum[i] = content_sum_dict[content_id]
    #         content_count[i] = content_count_dict[content_id]
            task_container_sum[i] = task_container_sum_dict[task_container_id]
            task_container_count[i] = task_container_count_dict[task_container_id]
            task_container_std[i]=task_container_std_dict[task_container_id]

            explanation_sum_dict[user_id] += prior_question_had_explanation
            explanation_count_dict[user_id] += 1
            explanation_sum[i] = explanation_sum_dict[user_id]
            explanation_count[i] = explanation_count_dict[user_id]

            if user_id in max_timestamp_u_dict['max_time_stamp'].keys():
                lagtime[i]=timestamp-max_timestamp_u_dict['max_time_stamp'][user_id]
                if(max_timestamp_u_dict2['max_time_stamp2'][user_id]==lagtime_mean2):#
                    lagtime2[i]=lagtime_mean2
                    lagtime3[i]=lagtime_mean3
                    #max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
                else:
                    lagtime2[i]=timestamp-max_timestamp_u_dict2['max_time_stamp2'][user_id]
                    if(max_timestamp_u_dict3['max_time_stamp3'][user_id]==lagtime_mean3):
                        lagtime3[i]=lagtime_mean3 #
                    else:
                        lagtime3[i]=timestamp-max_timestamp_u_dict3['max_time_stamp3'][user_id]
                    
                    max_timestamp_u_dict3['max_time_stamp3'][user_id]=max_timestamp_u_dict2['max_time_stamp2'][user_id]
                        
                max_timestamp_u_dict2['max_time_stamp2'][user_id]=max_timestamp_u_dict['max_time_stamp'][user_id]
                max_timestamp_u_dict['max_time_stamp'][user_id]=timestamp
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2
#                 lagtime_mean_dict[user_id]=lagtime_means[i]
            else:
                lagtime[i]=lagtime_mean
                max_timestamp_u_dict['max_time_stamp'].update({user_id:timestamp})
                lagtime2[i]=lagtime_mean2#
                max_timestamp_u_dict2['max_time_stamp2'].update({user_id:lagtime_mean2})
                lagtime3[i]=lagtime_mean3#
                max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
#                 lagtime_mean_dict.update({user_id:lagtime_mean})
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2

            if user_id in user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].keys():            
                delta_prior_question_elapsed_time[i]=prior_question_elapsed_time-user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]=prior_question_elapsed_time
            else:           
                delta_prior_question_elapsed_time[i]=delta_prior_question_elapsed_time_mean    
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].update({user_id:prior_question_elapsed_time})
            i=i+1 


        
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    #test_df = test_df[test_df['content_type_id'] == 0]
    #right_index=True
    #test_df = pd.merge(test_df, questions_df, on='content_id', how='left',right_index=True)    
    #test_df = pd.concat([test_df.reset_index(drop=True), questions_df.reindex(test_df['content_id'].values).reset_index(drop=True)], axis=1)
    test_df=test_df.merge(questions_df.loc[questions_df.index.isin(test_df['content_id'])],
                  how='left', on='content_id', right_index=True)
    
    #test_df = pd.merge(test_df, user_lecture_stats_part, on=['user_id'], how="left",right_index=True)
    #test_df = pd.concat([test_df.reset_index(drop=True), user_lecture_stats_part.reindex(test_df['user_id'].values).reset_index(drop=True)], axis=1)
#     test_df=test_df.merge(user_lecture_stats_part.loc[user_lecture_stats_part.index.isin(test_df['user_id'])],
#                   how='left', on='user_id', right_index=True)
 
    test_df['user_lecture_lv'] = user_lecture_sum / user_lecture_count
    test_df['user_lecture_sum'] = user_lecture_sum
    
    test_df['user_interaction_count'] = user_lecture_count
    test_df['user_interaction_timestamp_mean'] = test_df['timestamp']/user_lecture_count
    
    test_df['user_correctness'] = user_sum / user_count
    test_df['user_uncorrect_count'] =user_count-user_sum
    test_df['user_correct_count'] =user_sum
    test_df['true_user_correctness'] = (test_df['user_correct_count']+1)/(test_df['user_correct_count']/test_df['user_correctness'] + 2) 
    #test_df['user_answer_count'] =user_count
    
#     test_df['user_correctness2'] = user_sum2 / user_count2
#     test_df['user_uncorrect_count2'] =user_count2-user_sum2
#     test_df['user_correct_count2'] =user_sum2
    #test_df['user_answer_count2'] =user_count2
    
    #    
    test_df['task_container_correctness'] = task_container_sum / task_container_count
    test_df['task_container_cor_count'] = task_container_sum 
    test_df['task_container_uncor_count'] =task_container_count-task_container_sum 
    test_df['task_container_std'] = task_container_std 
    #test_df['content_task_mean'] = content_task_mean 
    
    test_df['explanation_mean'] = explanation_sum / explanation_count
    test_df['explanation_true_count'] = explanation_sum
    test_df['explanation_false_count'] = explanation_count-explanation_sum 
    
    #
    test_df['delta_prior_question_elapsed_time'] = delta_prior_question_elapsed_time 
    
  
 
    test_df["attempt_no"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
    test_df["lagtime"]=lagtime
    test_df["lagtime2"]=lagtime2
    test_df["lagtime3"]=lagtime3
    #test_df["lagtime_mean"]=lagtime_means

    test_df['timestamp']=test_df['timestamp']/(1000*3600)
    test_df.timestamp=test_df.timestamp.astype('float16')
    test_df['lagtime']=test_df['lagtime']/(1000*3600)
    test_df.lagtime=test_df.lagtime.astype('float32')
    test_df['lagtime2']=test_df['lagtime2']/(1000*3600)
    test_df.lagtime2=test_df.lagtime2.astype('float32')
    test_df['lagtime3']=test_df['lagtime3']/(1000*3600)
    test_df.lagtime3=test_df.lagtime3.astype('float32')
    test_df['user_interaction_timestamp_mean']=test_df['user_interaction_timestamp_mean']/(1000*3600)
    test_df.user_interaction_timestamp_mean=test_df.user_interaction_timestamp_mean.astype('float32')
    
    test_df['user_correctness'].fillna(0.67, inplace=True)
    
    test_preds  = model.predict(test_df[features])
    #######################
    #chess
    #######################
    
    if first_submission == False:
        last_df['answered_correctly'] = eval(test_df.iloc[0]['prior_group_answers_correct'])
        last_df['user_answer'] = eval(test_df.iloc[0]['prior_group_responses'])
        true_value.extend(eval(test_df.iloc[0]['prior_group_answers_correct']))
        for index, row in last_df.iterrows():
            user_pickle[row['user_id']].update_user(row)
    rows_accum = rows_accum + test_df.shape[0]
    if first_submission == False:
        1 + 1
       
    test_df['answered_correctly'] = 0.6524
    st = float(time.time())
    # 完成Merge 与 Concat 工作
    try:
        sub_1 = test_df[test_df['content_type_id'] == False]
        sub_2 = test_df[test_df['content_type_id'] == True]
        del test_df
        sub_1 = sub_1.merge(question_metadata, on="content_id", how="left")
        sub_2 = sub_2.merge(lesson_metadata, on="content_id", how="left")
        test_df = pd.DataFrame()
        test_df = pd.concat([sub_1, sub_2])
    except Exception:
        pass

    for index, row in test_df.iterrows():
        try:
            if row['user_id'] not in user_pickle:
                user_pickle[row['user_id']] = user()
            if row['content_type_id'] == 0:
                predict_dict = user_pickle[row['user_id']].process_output(row)
                l = []
                for i, v in predict_dict.items():
                    l.append(v)
                prd_value = float(model_chess.predict([l])[0])
                test_df.loc[test_df.row_id == row.row_id, 'answered_correctly'] = prd_value
                model_prd.append(prd_value)


        except Exception as e:
            print(e)
            pass

    time_taken = float(time.time()) - st

    if first_submission == True:
        first_submission = False
    last_df = test_df
    ######################
    
    
    
    
    
    test_df[target]=(np.array(outs) + np.array(test_preds) + np.array(prd_value))/3
    
    env.predict(test_df[['row_id', target]])

In [ ]:
sub = pd.read_csv('./submission.csv')
sub